### 简历项目1：使用DPO微调基于Qwen3-8B的推荐系统

#### 参考文献
Finetuning Large Language Model for Personalized Ranking https://arxiv.org/abs/2405.16127 <br><br>
#### 1. 前情摘要
#### 1.1 论文理解
&emsp;&emsp;在少样本或者零样本的情况下，将llm应用于推荐系统，但llm基于nlp任务设计，与推荐任务不同。SFT可以在训练过程中最大化生成正确答案的概率，当面对未见过的错误答案时，由于缺乏负样本学习，模型可能会高估其概率做出错误预测。所以，应该通过多个正负样本对之间进行学习。基于这种情况，在直接偏好优化（Direct Preference Optimization, DPO）的基础上，引入多个负样本的采样，扩大负样本的范围，建立更全面和平衡的正负样本关系。<br>

#### 1.2 BPR loss
 &emsp;&emsp;成对排序损失（Pair-Wise Ranking loss）是一种在机器学习中常用于排序任务的损失函数，它的目的是学习输入样本之间的相对距离，而不是直接预测一个标签或值，其中典型损失：贝叶斯个性化排序损失（Bayesian Personalized Ranking, BPR loss）。在推荐系统中，用户的历史行为数据通常是以隐式反馈形式存在的，例如用户的浏览、购买或点击行为。与显式反馈数据（例如用户的评分）相比，隐式反馈数据更加稀疏和难以解释。因此，推荐系统需要开发出适合隐式反馈数据的模型和算法来推荐物品。BPR损失函数就是为解决隐式反馈数据下的推荐问题而提出的。它的基本思想是：给定一个用户和两个物品，模型需要将用户更喜欢的物品排在用户更不喜欢的物品之前，从而学习到用户的个性化偏好。<br>

\begin{equation}
\mathcal{L}_{\mathrm{BPR}} = - \sum_{(u,i,j)\in\mathcal{D}} \ln \sigma\bigl(f_u(i) - f_u(j)\bigr)
\tag{1}
\end{equation}

 &emsp;&emsp; $\mathcal{D}$ 表示数据集, $u$ 表示用户, $i$ 和 $j$ 分别表示正样本物品与负样本物品。$ f_u(i)$ 表示用户 $u$ 对物品  $i$ 的偏好评分函数。常用的偏好评分函数包括余弦相似度（cosine similarity）和皮尔逊相关系数（Pearson correlation coefficient）。通过优化 BPR 损失函数，可以提高用户 $u$ 对正样本物品 $i$ 的偏好得分 $ f_u(i)$ ，同时降低其对负样本物品 $j$  的偏好得分 $ f_u(j)$。
#### 2. 总体步骤
<div align="center">
   <img src="./pics/rp1_dpo_1.png" width="800" />
</div>

#### 2.1 任务描述
 &emsp;&emsp;论文提出的DMPO 旨在判断用户是否喜欢所给物品，该任务本质上属于二分类任务。用户的物品偏好历史会被输入到提示中，随后从用户-物品交互列表中选取一对项目来进行预测。其中一项记为 $i$ 正样本，已被用户给予较高的评分，且高于3分（总分5分）。另一项记为负样本 $j$，被用户给予的评分较低，低于 3 分。该模型会通过生成一个推荐列表，该列表会根据用户对各项给予高评分的可能性进行排序，并将项目 $i$ 排在项目 $j$ 之前，从而确定用户更有可能喜欢正样本项目 $i$ 而非负样本项目 $j$。

#### 2.2 SFT和DPO阶段的提示词




**数据集**：Amazon Movies and TV，<br>